In [1]:
# SCREEN TIME PREDICTION MODEL
# Model: MLPRegressor (Neural Network)
# Input: User's 7 days of screen time (hours)
# Output: Predicted Expected_Time and Healthy_Time (hours)

# --- 1. Import Required Libraries ---
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error, r2_score
from IPython.display import display

# --- 2. Generate Synthetic "Realistic" Dataset ---

num_users = 2000
data = []

# Define feature and target names
feature_cols = [f'Day{i}' for i in range(1, 8)]
target_cols = ['Expected_Time', 'Healthy_Time'] # Renamed to 'Healthy_Time'

for _ in range(num_users):
    # Create different user profiles for realism
    profile = np.random.choice(['stable', 'increasing', 'decreasing', 'chaotic'], p=[0.5, 0.2, 0.2, 0.1])

    if profile == 'stable':
        base_usage = np.random.uniform(2.0, 5.0)
        days_input = np.random.normal(base_usage, 0.5, 7)
    elif profile == 'increasing':
        start_usage = np.random.uniform(2.0, 4.0)
        trend = np.random.uniform(0.2, 0.5)
        days_input = [start_usage + (i * trend) + np.random.normal(0, 0.4) for i in range(7)]
    elif profile == 'decreasing':
        start_usage = np.random.uniform(6.0, 10.0)
        trend = np.random.uniform(-0.3, -0.6)
        days_input = [start_usage + (i * trend) + np.random.normal(0, 0.5) for i in range(7)]
    else:
        days_input = np.random.uniform(1.0, 9.0, 7)

    days_input = np.clip(days_input, 0.5, 16.0)

    # --- Create Target Variables (The "Logic" the Model Must Learn) ---
    mean_usage = np.mean(days_input)
    trend = days_input[-1] - days_input[0]

    # 1. Expected_Time
    noise = np.random.normal(0, 0.25)
    expected_time = mean_usage + (trend * 0.2) + noise

    # 2. Healthy_Time (formerly Safe_Time)
    baseline_healthy = 2.5
    healthy_time = baseline_healthy - (mean_usage - 4.0) * 0.4 + np.random.normal(0, 0.2)

    # Clip targets
    expected_time = np.clip(expected_time, 0.5, 16.0)
    healthy_time = np.clip(healthy_time, 0.5, 4.0) # Healthy time is capped lower

    row = list(days_input) + [expected_time, healthy_time]
    data.append(row)

# Create DataFrame
columns = feature_cols + target_cols
df = pd.DataFrame(data, columns=columns)

print(f"✅ Dataset generated with {num_users} users.")
print("--- Sample of Generated Data (Top 20 Users) ---")
# [CORRECTION 1] Changed df.head() to df.head(20)
display(df.head(20))

# --- 3. Define Features (X) and Targets (y) ---
X = df[feature_cols]
y = df[target_cols]

# --- 4. Split Data into Training and Test Sets ---
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print(f"\nData split: {len(X_train)} training samples, {len(X_test)} testing samples.")

# --- 5. Preprocessing & Scaling (CRITICAL for Neural Networks) ---
x_scaler = StandardScaler()
X_train_scaled = x_scaler.fit_transform(X_train)
X_test_scaled = x_scaler.transform(X_test)

y_scaler = StandardScaler()
y_train_scaled = y_scaler.fit_transform(y_train)

print("\n✅ Features and Targets scaled.")

# --- 6. Train the MLPRegressor (Neural Network) Model ---
print("--- Training Neural Network (MLPRegressor) ---")

model = MLPRegressor(
    hidden_layer_sizes=(64, 32),
    activation='relu',
    solver='adam',
    max_iter=500,
    random_state=42,
    alpha=0.001,
    early_stopping=True,
    validation_fraction=0.1,
    verbose=False
)

model.fit(X_train_scaled, y_train_scaled)
print("✅ Model trained successfully.")

# --- 7. Evaluate the Model ---
print("\n--- Model Evaluation ---")
y_pred_scaled = model.predict(X_test_scaled)
y_pred = y_scaler.inverse_transform(y_pred_scaled)

# Evaluate 'Expected_Time'
r2_expected = r2_score(y_test['Expected_Time'], y_pred[:, 0])
rmse_expected = np.sqrt(mean_squared_error(y_test['Expected_Time'], y_pred[:, 0]))
print(f"🎯 Target 1: Expected_Time")
print(f"   R² Score: {r2_expected:.4f}")
print(f"   RMSE:     {rmse_expected:.4f} hours")

# Evaluate 'Healthy_Time'
r2_healthy = r2_score(y_test['Healthy_Time'], y_pred[:, 1])
rmse_healthy = np.sqrt(mean_squared_error(y_test['Healthy_Time'], y_pred[:, 1]))
print(f"\n🎯 Target 2: Healthy_Time")
print(f"   R² Score: {r2_healthy:.4f}")
print(f"   RMSE:     {rmse_healthy:.4f} hours")

# --- 8. Interactive Prediction ---
# [CORRECTION 2] Added interactive input section as requested
print("\n" + "="*40)
print("🔮 INTERACTIVE SCREEN TIME PREDICTION")
print("="*40)
print("\nEnter your average screen time (in hours) for the last 7 days.")

user_input = []
for i in range(1, 8):
    while True:
        try:
            # Prompt user for input
            time_str = input(f"  ➡ Day {i}: ")
            time_float = float(time_str)

            # Validate input
            if 0.0 <= time_float <= 24.0:
                user_input.append(time_float)
                break # Exit while loop and go to next day
            else:
                print("   Error: Please enter a value between 0 and 24.")
        except ValueError:
            print("   Error: Invalid input. Please enter a number (e.g., 4.5).")

print("\nProcessing your data...")

# Convert user's list into a 2D NumPy array for the model
user_data_np = np.array([user_input])

# 1. Scale the user's data using the FITTED x_scaler
user_data_scaled = x_scaler.transform(user_data_np)

# 2. Predict using the trained model
user_pred_scaled = model.predict(user_data_scaled)

# 3. Inverse transform the prediction using the FITTED y_scaler
user_pred = y_scaler.inverse_transform(user_pred_scaled)

# 4. Display the final, user-friendly results
print("\n--- 📈 Your Personalized Prediction 💚 ---")
print(f"  Your Input (Last 7 Days): {user_input}")
print("-" * 40)
print(f"  📈 Predicted Expected Screen Time: {user_pred[0, 0]:.2f} hours")
print(f"  💚 Recommended Healthy Screen Time:  {user_pred[0, 1]:.2f} hours")
print("="*40)

/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:1256: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. Use OneVsRestClassifier(LogisticRegression(..)) instead. Leave it to its default value to avoid this warning.
  warnings.warn(
